<a href="https://colab.research.google.com/github/AlyonaSarapina/python-and-cyber/blob/main/CDProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

As a part of the final project we were creating a solution for when people need to relocate to a new area and need to find schools within a reasonable distance of where they would live. This might determine their choice of location.

In [14]:
#aditional functions for Googgle maps API
!pip install googlemaps

In [1]:
import os
from google.colab import output

google_api_key = input("Enter Google API key: ")
aws_api_key = input("Enter AWS API key: ")
os.environ['AWS_API_KEY'] = aws_api_key
os.environ['GOOGLE_API_KEY'] = google_api_key
output.clear()

AS a user WHEN I want to find the nearest schools to a given location THEN I will be able to run a function in a Google Colab that will ask Google maps to send a list of nearby schools AND I will be shown a list of schools in that area AND this information can also be sent to a serverless function on AWS and saved in a schools file on S3

In [29]:
import googlemaps

gmaps = googlemaps.Client(os.environ.get('GOOGLE_API_KEY'))

def get_schools(lat, lon, rad = 1):
  school_list=[]
  try:
    places_result  = gmaps.places_nearby(location= f"{lat},{lon}", radius = rad, type = "school")
    for place in places_result['results']:
      school_name= place['name']
      school_location_lat= place['geometry']['location']['lat']
      school_location_lng= place['geometry']['location']['lng']
      school_list.append([school_name, school_location_lat,school_location_lng])
    if len(school_list) == 0:
      return "No schools in this area"
    else:
      return school_list
  except googlemaps.exceptions.ApiError as e:
    if e.status=="INVALID_REQUEST":
      return "coordinates are invalid"
  except Exception as e:
    return "Error: " + str(e)

In [ ]:
#Function that will save the list as txt.file ans send it to the bucket (aws)

In [ ]:
import requests
import json

def send_post_request(array):
  try:
      url = "https://3ucizs3hm4.execute-api.eu-west-2.amazonaws.com/default/myThirdLambdaFunction"

      payload = json.dumps({
        "data": array
      })

      headers = {
        'Content-Type': 'application/json',
        'X-Api-Key': os.environ.get('AWS_API_KEY')
      }

      response = requests.post(url, headers=headers, data=payload)

      print(response.text)
  except Exception as e:
    return "Error: " + str(e)

result = get_schools(51.5072, 0.1276, 100000)
send_post_request(result)

{"message": "New file been successfully created", "data": [["Haling Manor High School", 51.3533553, -0.1092771], ["Lambeth School of Art", 51.4871005, -0.1185023], ["University of Roehampton London", 51.4563401, -0.2446496], ["Nuffield Health City Fitness & Wellbeing Gym", 51.5099867, -0.09127799999999998], ["IOE, UCL's Faculty of Education and Society", 51.5229371, -0.1279417], ["Newham Sixth Form College", 51.5224054, 0.03081829999999999], ["Richmond upon Thames College", 51.4517137, -0.3425717], ["Whitgift School", 51.36002, -0.1022125], ["City of Westminster College, Paddington Green Campus", 51.52104159999999, -0.1746649], ["South Hampstead High School", 51.5461891, -0.1769548], ["Eltham College", 51.43804009999999, 0.038777], ["Kingston Grammar School", 51.41025389999999, -0.2964765], ["Harrow School", 51.5725237, -0.3334223000000001], ["St Dominic's Sixth Form College", 51.5663649, -0.3415743000000001], ["Queen Mother Sports Centre", 51.493354, -0.1405451], ["Paddington Academy"

In [ ]:
#function that will send the list to the bucket

AS a user WHEN I want to see all the schools in the file on AWS S3 THEN I will be able to run a function in a Google Colab that will send a request to get the full list, AND I will see the list printed in my output

In [25]:
import requests

def send_get_request():
  url = "https://3ucizs3hm4.execute-api.eu-west-2.amazonaws.com/default/myThirdLambdaFunction"
  payload = {}
  headers = {
    'x-api-key': os.environ.get('AWS_API_KEY')
  }

  response = requests.request("GET", url, headers=headers, data=payload)

  print(response.text)

send_get_request()


{"message": "The data has been found", "data": [["0", "1", "2"], ["Bannockburn Primary School & Nursery", "51.4869172", "0.1015561"], ["St Margaret Clitherow Primary School", "51.50103289999999", "0.1132992"]]}


ACCEPTANCE CRITERIA - USER STORIES

WHEN I send latitude 55.816555 and longitude -4.309890 and a radius of 4km THEN the following list is added to the file:

In [26]:
result= get_schools(55.816555, -4.309890,4000)
print(result)

[["Hutchesons' Grammar School", 55.83644289999999, -4.282187], ['Eastwood High School', 55.78701009999999, -4.327643299999999], ['Glasgow Club Bellahouston', 55.8459204, -4.3239934], ['Bellahouston Academy', 55.8481055, -4.3005388], ['Pro-Soccer', 55.7979717, -4.3199578], ['St Vincent’s Primary School', 55.8102127, -4.3248058], ['Cleeves Primary School', 55.81444250000001, -4.355818899999999], ['Thornliebank Primary School', 55.8093009, -4.3143177], ['Our Lady of the Mission Primary School', 55.8017274, -4.307100999999999], ['Hillpark Autism Unit', 55.8155973, -4.297950300000001], ['Fotheringay Centre', 55.8371502, -4.2833944], ['Animal Man Parties', 55.8158912, -4.295114700000001], ['Woodland Outdoor Kindergartens', 55.8179202, -4.301552699999999], ['Kumon Maths & English', 55.80408809999999, -4.2935565], ['Carmichael Nursery School', 55.8245123, -4.272025], ['Wendy King', 55.8293055, -4.2910509], ["Burnbrae Children's Centre", 55.81449809999999, -4.3556976], ['Zippytotz', 55.8099591,

WHEN I have entered a latitude of 87.543255 and longitude of 0, and a radius of 10km THEN I will receive an empty list and nothing will be saved

In [27]:
result= get_schools(87.543255, 0,10000)
print(result)

No schools in this area


WHEN I have entered a latitude of 99.999995 and longitude of 0, and a radius of 5km THEN I will receive and error message telling me that the latitude is invalid (too big)

In [30]:
result= get_schools(99.999995, 0,50000)
print(result)

coordinates are invalid
